In [1]:
import pandas as pd

# Cargar el archivo Excel
file_path = r"C:\Users\joaqu\Downloads\x0.xlsx"
df = pd.read_excel(file_path)

# Análisis descriptivo inicial
def describe_data(df):
    print("Resumen del dataframe:")
    print(df.info())
    print("\nEstadísticas descriptivas:\n")
    print(df.describe(include='all').transpose())

    print("\nValores únicos por columna:\n")
    unique_counts = df.nunique()
    print(unique_counts)

    print("\nPorcentaje de valores nulos por columna:\n")
    null_percentage = df.isnull().mean() * 100
    print(null_percentage)

    print("\nColumnas con alta varianza:\n")
    varianza = df.var(numeric_only=True)
    high_variance = varianza[varianza > varianza.mean()]
    print(high_variance)

    print("\nColumnas con pocos valores únicos:\n")
    low_unique = unique_counts[unique_counts <= 5]
    print(low_unique)

    return unique_counts, null_percentage, high_variance, low_unique

# Identificar columnas potencialmente problemáticas
def identify_columns_to_remove(unique_counts, null_percentage, high_variance, low_unique):
    columns_to_remove = []

    # Columnas con más del 50% de valores nulos
    null_threshold = 50
    columns_to_remove += list(null_percentage[null_percentage > null_threshold].index)

    # Columnas con un único valor
    columns_to_remove += list(unique_counts[unique_counts <= 1].index)

    # Columnas categóricas con baja diversidad
    columns_to_remove += list(low_unique.index)

    # Eliminar duplicados en la lista
    columns_to_remove = list(set(columns_to_remove))

    return columns_to_remove

# Ejecución
def main():
    print("Analizando el dataset...\n")
    unique_counts, null_percentage, high_variance, low_unique = describe_data(df)

    print("\nIdentificando columnas a eliminar...\n")
    columns_to_remove = identify_columns_to_remove(unique_counts, null_percentage, high_variance, low_unique)
    
    print(f"Columnas sugeridas para eliminar ({len(columns_to_remove)}):")
    print(columns_to_remove)

if __name__ == "__main__":
    main()

Analizando el dataset...

Resumen del dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44296 entries, 0 to 44295
Data columns (total 78 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   borrow_block_number                      44296 non-null  int64  
 1   borrow_timestamp                         44296 non-null  int64  
 2   wallet_address                           44296 non-null  object 
 3   first_tx_timestamp                       44296 non-null  int64  
 4   last_tx_timestamp                        44296 non-null  int64  
 5   wallet_age                               44296 non-null  int64  
 6   incoming_tx_count                        44296 non-null  int64  
 7   outgoing_tx_count                        44296 non-null  int64  
 8   net_incoming_tx_count                    44296 non-null  int64  
 9   total_gas_paid_eth                       44296 non-null  float64
 1

In [2]:
# Eliminar columnas relacionadas con "market" e identificadores irrelevantes
def clean_data(df):
    # Identificar columnas relacionadas con "market" y eliminar
    market_cols = [col for col in df.columns if "market" in col]
    identifier_cols = ["wallet_address", "borrow_block_number", "borrow_timestamp"]
    columns_to_drop = market_cols + identifier_cols
    df_cleaned = df.drop(columns=columns_to_drop, errors="ignore")
    
    print(f"Columnas eliminadas ({len(columns_to_drop)}): {columns_to_drop}")
    
    # Transformar columnas temporales en días
    temporal_cols = [col for col in df_cleaned.columns if "timestamp" in col]
    for col in temporal_cols:
        df_cleaned[col + "_days"] = (df_cleaned[col] - df_cleaned[col].min()) / (24 * 60 * 60)
        df_cleaned = df_cleaned.drop(columns=[col])  # Eliminar la columna original
    
    print(f"Columnas temporales transformadas: {temporal_cols}")
    
    return df_cleaned

# Ejecutar el proceso de limpieza
df_cleaned = clean_data(df)

# Verificar la estructura después de la limpieza
print("Estructura del DataFrame limpio:")
print(df_cleaned.info())

# Exportar el DataFrame limpio si es necesario
df_cleaned.to_csv("cleaned_dataset.csv", index=False)


Columnas eliminadas (30): ['market_adx', 'market_adxr', 'market_apo', 'market_aroonosc', 'market_aroonup', 'market_atr', 'market_cci', 'market_cmo', 'market_correl', 'market_dx', 'market_fastk', 'market_fastd', 'market_ht_trendmode', 'market_linearreg_slope', 'market_macd_macdext', 'market_macd_macdfix', 'market_macd', 'market_macdsignal_macdext', 'market_macdsignal_macdfix', 'market_macdsignal', 'market_max_drawdown_365d', 'market_natr', 'market_plus_di', 'market_plus_dm', 'market_ppo', 'market_rocp', 'market_rocr', 'wallet_address', 'borrow_block_number', 'borrow_timestamp']
Columnas temporales transformadas: ['first_tx_timestamp', 'last_tx_timestamp', 'risky_first_tx_timestamp', 'risky_last_tx_timestamp', 'risky_first_last_tx_timestamp_diff']
Estructura del DataFrame limpio:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44296 entries, 0 to 44295
Data columns (total 48 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                     

In [ ]:
correlations = df_cleaned.corr()['target'].sort_values()
low_correlation_cols = correlations[abs(correlations) < 0.05].index.tolist()
print("Columnas con baja correlación con target:", low_correlation_cols)


In [ ]:
description = df_cleaned.describe().transpose()
print(description[['min', 'max', 'mean', 'std']])


In [ ]:
zero_percentage = (df_cleaned == 0).mean() * 100
high_zero_cols = zero_percentage[zero_percentage > 75].index.tolist()
print("Columnas con más del 95% de ceros:", high_zero_cols)


In [3]:
columns_to_keep = [
    'incoming_tx_count', 'outgoing_tx_count', 'net_incoming_tx_count',
    'risky_tx_count', 'outgoing_tx_sum_eth', 'incoming_tx_sum_eth',
    'total_balance_eth', 'risk_factor', 'total_collateral_eth',
    'total_available_borrows_eth', 'total_available_borrows_avg_eth',
    'avg_weighted_risk_factor', 'risk_factor_above_threshold_daily_count',
    'avg_risk_factor', 'max_risk_factor', 'borrow_amount_sum_eth',
    'borrow_amount_avg_eth', 'borrow_count', 'repay_count',
    'borrow_repay_diff_eth', 'deposit_count', 'deposit_amount_sum_eth',
    'withdraw_amount_sum_eth', 'target'
]

# Crear un nuevo DataFrame manteniendo solo las columnas deseadas
df_filtered = df_cleaned[columns_to_keep]

# Verificar la estructura del DataFrame filtrado
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44296 entries, 0 to 44295
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   incoming_tx_count                        44296 non-null  int64  
 1   outgoing_tx_count                        44296 non-null  int64  
 2   net_incoming_tx_count                    44296 non-null  int64  
 3   risky_tx_count                           44296 non-null  int64  
 4   outgoing_tx_sum_eth                      44296 non-null  float64
 5   incoming_tx_sum_eth                      44296 non-null  float64
 6   total_balance_eth                        44296 non-null  float64
 7   risk_factor                              44296 non-null  float64
 8   total_collateral_eth                     44296 non-null  float64
 9   total_available_borrows_eth              44296 non-null  float64
 10  total_available_borrows_avg_eth          44296

In [4]:
import pandas as pd

# Cargar el dataset adicional
file_path_t1 = r"C:\Users\joaqu\Downloads\XX0.xlsx"
data_t1 = pd.read_excel(file_path_t1)

# Función para limpiar el dataset, eliminando columnas irrelevantes y transformando temporales
def clean_data(df):
    # Identificar columnas relacionadas con "market" y eliminar
    market_cols = [col for col in df.columns if "market" in col]
    identifier_cols = ["wallet_address", "borrow_block_number", "borrow_timestamp"]
    columns_to_drop = market_cols + identifier_cols
    df_cleaned = df.drop(columns=columns_to_drop, errors="ignore")

    print(f"Columnas eliminadas ({len(columns_to_drop)}): {columns_to_drop}")

    # Transformar columnas temporales en días
    temporal_cols = [col for col in df_cleaned.columns if "timestamp" in col]
    for col in temporal_cols:
        df_cleaned[col + "_days"] = (df_cleaned[col] - df_cleaned[col].min()) / (24 * 60 * 60)
        df_cleaned = df_cleaned.drop(columns=[col], errors="ignore")  # Eliminar la columna original

    print(f"Columnas temporales transformadas: {temporal_cols}")

    return df_cleaned

# Limpiar el dataset adicional
data_t1_cleaned = clean_data(data_t1)

# Seleccionar las columnas finales a mantener
columns_to_keep = [
    'incoming_tx_count', 'outgoing_tx_count', 'net_incoming_tx_count',
    'risky_tx_count', 'outgoing_tx_sum_eth', 'incoming_tx_sum_eth',
    'total_balance_eth', 'risk_factor', 'total_collateral_eth',
    'total_available_borrows_eth', 'total_available_borrows_avg_eth',
    'avg_weighted_risk_factor', 'risk_factor_above_threshold_daily_count',
    'avg_risk_factor', 'max_risk_factor', 'borrow_amount_sum_eth',
    'borrow_amount_avg_eth', 'borrow_count', 'repay_count',
    'borrow_repay_diff_eth', 'deposit_count', 'deposit_amount_sum_eth',
    'withdraw_amount_sum_eth', 'target'
]

data_t1_filtered = data_t1_cleaned[columns_to_keep]

# Separar características y target
X_t1 = data_t1_filtered.drop(columns=['target'])
y_t1 = data_t1_filtered['target']

# Confirmar que las columnas son correctas
assert list(X_t1.columns) == columns_to_keep[:-1], "Las columnas no coinciden exactamente."

print("El dataset adicional ha sido preprocesado correctamente y está listo para usar.")

Columnas eliminadas (30): ['market_adx', 'market_adxr', 'market_apo', 'market_aroonosc', 'market_aroonup', 'market_atr', 'market_cci', 'market_cmo', 'market_correl', 'market_dx', 'market_fastk', 'market_fastd', 'market_ht_trendmode', 'market_linearreg_slope', 'market_macd_macdext', 'market_macd_macdfix', 'market_macd', 'market_macdsignal_macdext', 'market_macdsignal_macdfix', 'market_macdsignal', 'market_max_drawdown_365d', 'market_natr', 'market_plus_di', 'market_plus_dm', 'market_ppo', 'market_rocp', 'market_rocr', 'wallet_address', 'borrow_block_number', 'borrow_timestamp']
Columnas temporales transformadas: ['first_tx_timestamp', 'last_tx_timestamp', 'risky_first_tx_timestamp', 'risky_last_tx_timestamp', 'risky_first_last_tx_timestamp_diff']
El dataset adicional ha sido preprocesado correctamente y está listo para usar.


In [6]:
# Comparar columnas entre data_t1_filtered y el dataset original (data)
original_columns = set(df_filtered.columns)  # Asumiendo que `data` es el dataset original limpio
filtered_columns = set(data_t1_filtered.columns)

# Identificar diferencias
missing_in_filtered = original_columns - filtered_columns
extra_in_filtered = filtered_columns - original_columns

# Mostrar resultados de comparación
print("Columnas en el dataset original pero no en el filtrado:", missing_in_filtered)
print("Columnas en el dataset filtrado pero no en el original:", extra_in_filtered)

# Verificar si son exactamente iguales
if original_columns == filtered_columns:
    print("El dataset procesado tiene exactamente las mismas columnas que el original.")
else:
    print("El dataset procesado tiene diferencias en las columnas.")


Columnas en el dataset original pero no en el filtrado: set()
Columnas en el dataset filtrado pero no en el original: set()
El dataset procesado tiene exactamente las mismas columnas que el original.


In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split
import optuna
from xgboost import XGBClassifier
import lightgbm as lgb

# Cargar el dataset limpio
data = df_filtered.copy()

# Separar características y target
X = data.drop(columns=['target'])
y = data['target']

# División de datos en entrenamiento (70%), validación (20%) y prueba (10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42, stratify=y_temp)

# Definir función de evaluación AUC-PR
def calculate_auc_pr(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    return auc(recall, precision)

# Función de optimización para LightGBM
def objective_lightgbm(trial):
    params = {
        'objective': 'binary',
        'metric': 'aucpr',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 250),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 25)
    }

    train_data = lgb.Dataset(X_train, label=y_train)
    model = lgb.train(params, train_data, num_boost_round=1500, valid_sets=[lgb.Dataset(X_val, label=y_val)], 
                      )
    preds = model.predict(X_val)
    return calculate_auc_pr(y_val, preds)

# Función de optimización para XGBoost
def objective_xgboost(trial):
    params = {
        'objective': 'binary:logistic',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.25),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'gamma': trial.suggest_loguniform('gamma', 0.01, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.8, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0)
    }

    model = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
    preds = model.predict_proba(X_val)[:, 1]
    return calculate_auc_pr(y_val, preds)

# Optimización con Optuna
print("Optimizando LightGBM...")
study_lightgbm = optuna.create_study(direction='maximize', study_name='LightGBM Optimization')
study_lightgbm.optimize(objective_lightgbm, n_trials=25)

print("Optimizando XGBoost...")
study_xgboost = optuna.create_study(direction='maximize', study_name='XGBoost Optimization')
study_xgboost.optimize(objective_xgboost, n_trials=25)

# Resultados
results = {
    'Model': ['LightGBM', 'XGBoost'],
    'Best AUC-PR': [
        study_lightgbm.best_trial.value,
        study_xgboost.best_trial.value,
    ],
    'Best Params': [
        study_lightgbm.best_trial.params,
        study_xgboost.best_trial.params,
    ]
}

results_df = pd.DataFrame(results)
print(results_df)

[I 2024-12-11 12:16:57,492] A new study created in memory with name: LightGBM Optimization


Optimizando LightGBM...
[LightGBM] [Info] Number of positive: 18124, number of negative: 17312
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002008 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5686
[LightGBM] [Info] Number of data points in the train set: 35436, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.511457 -> initscore=0.045837
[LightGBM] [Info] Start training from score 0.045837


C:\Users\joaqu\AppData\Local\Temp\ipykernel_15900\329155175.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.3),
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15900\329155175.py:34: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15900\329155175.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-12-11 12:17:04,774] Trial 0 finished with value: 0.962467129061681 and parameters: {'learning_rate': 0.2601656358747541, 'num_leaves': 239, 'feature_fraction': 0.5355813780543405, 'bagging_fraction': 0.7799200682948355, 'bagging_freq': 3, 'min_data_in_leaf': 54, 'max_depth': 25}. Best is trial 0 with value: 0.962467129061681.
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15900\329155175.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.3),
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15900\329155175.py:34: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 

...

[1]	validation_0-logloss:0.67640
[2]	validation_0-logloss:0.66839
[3]	validation_0-logloss:0.66088
[4]	validation_0-logloss:0.65349
[5]	validation_0-logloss:0.64620
[6]	validation_0-logloss:0.63931
[7]	validation_0-logloss:0.63264
[8]	validation_0-logloss:0.62610
[9]	validation_0-logloss:0.61955
[10]	validation_0-logloss:0.61328
[11]	validation_0-logloss:0.60722
[12]	validation_0-logloss:0.60141
[13]	validation_0-logloss:0.59573
[14]	validation_0-logloss:0.59011
[15]	validation_0-logloss:0.58481
[16]	validation_0-logloss:0.57970
[17]	validation_0-logloss:0.57447
[18]	validation_0-logloss:0.56933
[19]	validation_0-logloss:0.56429
[20]	validation_0-logloss:0.55954
[21]	validation_0-logloss:0.55495
[22]	validation_0-logloss:0.55051
[23]	validation_0-logloss:0.54619
[24]	validation_0-logloss:0.54170
[25]	validation_0-logloss:0.53752
[26]	validation_0-logloss:0.53338
[27]	validation_0-logloss:0.52935
[28]	validation_0-logloss:0.52535
[29]	validation_0-logloss:0.52151
[30]	validation_0-loglo

[I 2024-12-11 12:27:30,742] Trial 23 finished with value: 0.965634346657093 and parameters: {'learning_rate': 0.017195596933632835, 'max_depth': 12, 'n_estimators': 1475, 'gamma': 0.0288613105707185, 'subsample': 0.941459826884955, 'colsample_bytree': 0.8595338121563129}. Best is trial 11 with value: 0.966527398054741.
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15900\329155175.py:51: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.25),
C:\Users\joaqu\AppData\Local\Temp\ipykernel_15900\329155175.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial.suggest_loguniform('gamma', 0.01, 1.0),
C:\

[0]	validation_0-logloss:0.67956
[1]	validation_0-logloss:0.66689
[2]	validation_0-logloss:0.65461
[3]	validation_0-logloss:0.64304
[4]	validation_0-logloss:0.63213
[5]	validation_0-logloss:0.62158
[6]	validation_0-logloss:0.61174
[7]	validation_0-logloss:0.60226
[8]	validation_0-logloss:0.59303
[9]	validation_0-logloss:0.58418
[10]	validation_0-logloss:0.57587
[11]	validation_0-logloss:0.56751
[12]	validation_0-logloss:0.55963
[13]	validation_0-logloss:0.55188
[14]	validation_0-logloss:0.54444
[15]	validation_0-logloss:0.53730
[16]	validation_0-logloss:0.53079
[17]	validation_0-logloss:0.52430
[18]	validation_0-logloss:0.51817
[19]	validation_0-logloss:0.51185
[20]	validation_0-logloss:0.50585
[21]	validation_0-logloss:0.50023
[22]	validation_0-logloss:0.49483
[23]	validation_0-logloss:0.48953
[24]	validation_0-logloss:0.48436
[25]	validation_0-logloss:0.47946
[26]	validation_0-logloss:0.47458
[27]	validation_0-logloss:0.46996
[28]	validation_0-logloss:0.46550
[29]	validation_0-loglos

[I 2024-12-11 12:27:56,857] Trial 24 finished with value: 0.965788170265552 and parameters: {'learning_rate': 0.026700884887790134, 'max_depth': 14, 'n_estimators': 1720, 'gamma': 0.023884392793621912, 'subsample': 0.9724434960735108, 'colsample_bytree': 0.9980369322269169}. Best is trial 11 with value: 0.966527398054741.


      Model  Best AUC-PR                                        Best Params
0  LightGBM     0.967201  {'learning_rate': 0.2089785172663563, 'num_lea...
1   XGBoost     0.966527  {'learning_rate': 0.026249593044418627, 'max_d...


In [11]:
# Resultados
results = {
    'Model': ['LightGBM', 'XGBoost'],
    'Best AUC-PR': [
        study_lightgbm.best_trial.value,
        study_xgboost.best_trial.value,
    ],
    'Best Params': [
        study_lightgbm.best_trial.params,
        study_xgboost.best_trial.params,
    ]
}

results_df = pd.DataFrame(results)
print(results_df)

      Model  Best AUC-PR                                        Best Params
0  LightGBM     0.967201  {'learning_rate': 0.2089785172663563, 'num_lea...
1   XGBoost     0.966527  {'learning_rate': 0.026249593044418627, 'max_d...
